## Load from CVS
The code in this notebook copies the cvs files for beaked whales from S3 to HDFS and then loads the data into 
a spark DataFrame.


In [3]:
from pyspark import SparkContext
sc = SparkContext(master=master_url)

from pyspark.sql import Row, SQLContext,DataFrame
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

%pylab inline

!pip install pandas
!pip install scipy

import pandas as pd
import datetime as dt

from scipy.io import loadmat,savemat,whosmat

from string import split
from collections import Counter
import re
import numpy as np
from numpy import shape

from glob import glob

### Format of cvs files
|field name     | Description               | Data Type
|---------------|---------------------------|--------------
|0: time        | time of click             | string in datetime format `%Y-%m-%d %H:%M:%S.%f`
|1: species		| Initial species classification	        | 'str'
|2: site		| name of site		        | 'str'
|3: rec_no		| recording number		    | 'str'
|4: bout_i		| bout number		        | numpy.int64
|5: peak2peak	| peak to peak magnitude    | 			numpy.float64
|6: MSN	        |	wave form |		 an array of length 202
|208: MSP		|	spectra |	 an array of length 101  
|309: TPWS1		| 1 if click appears in TPWS1	| 	bool
|310: MD1		|	--- " ---	in MD1|	bool
|311: FD1	    |	--- " ---	in FD1|	bool
|312: TPWS2		| 1 if click appears in TPWS2	| 	bool
|313: MD2		|	--- " ---	in MD2|	bool
|314: FD2	    |	--- " ---	in FD2|	bool
|315: TPWS3		| 1 if click appears in TPWS3	| 	bool
|316: MD3		|	--- " ---	in MD3|	bool
|317: FD3	    |	--- " ---	in FD3|	bool
total number of fields= 318


### Parsing code

In [ ]:
# %load lib/row_parser.py

In [17]:
import sys
sys.path.append('lib')
from row_parser import row_parser
Parser=row_parser()
#Parser.parse('lahksjdksj')

('time', 'datetime') {'start': 0, 'parser': <function parse_date at 0x1086d8de8>, 'end': 1, 'name': 'time'}
('species', 'str') {'start': 1, 'parser': <function parse_string at 0x1086cc488>, 'end': 2, 'name': 'species'}
('site', 'str') {'start': 2, 'parser': <function parse_string at 0x1086cc488>, 'end': 3, 'name': 'site'}
('rec_no', 'str') {'start': 3, 'parser': <function parse_string at 0x1086cc488>, 'end': 4, 'name': 'rec_no'}
('bout_i', 'int') {'start': 4, 'parser': <function parse_int at 0x1086bbc80>, 'end': 5, 'name': 'bout_i'}
('peak2peak', 'float') {'start': 5, 'parser': <function parse_float at 0x1086bb320>, 'end': 6, 'name': 'peak2peak'}
('MSN', 'array', 202) {'start': 6, 'parser': <function parse_array at 0x1086bbed8>, 'end': 208, 'name': 'MSN'}
('MSP', 'array', 101) {'start': 208, 'parser': <function parse_array at 0x1086bbed8>, 'end': 309, 'name': 'MSP'}
('TPWS1', 'bool') {'start': 309, 'parser': <function parse_int at 0x1086bbc80>, 'end': 310, 'name': 'TPWS1'}
('MD1', 'boo

### Code for 

In [12]:
%cd /root/ipython/BeakedWhaleClassification/
%run /root/Credentials.ipynb

/root/ipython/BeakedWhaleClassification


In [13]:
s3helper.set_credential(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

In [14]:
s3helper.open_bucket('while-classification')
s3helper.ls_s3()

[u'CVS',
 u'DT_Cuviers',
 u'DT_Gervais',
 u'GC_Cuviers',
 u'GC_Gervais',
 u'MC_Cuviers',
 u'MC_Gervais']

In [15]:
dirs=s3helper.ls_s3('CVS')
dirs[:10]

[u'CVS/DT.01.Cuviers',
 u'CVS/DT.01.Gervais',
 u'CVS/DT.02.Cuviers',
 u'CVS/DT.02.Gervais',
 u'CVS/DT.03.Cuviers',
 u'CVS/DT.03.Gervais',
 u'CVS/DT.04.Cuviers',
 u'CVS/DT.04.Gervais',
 u'CVS/DT.05.Cuviers',
 u'CVS/DT.05.Gervais']

In [16]:
from time import time

### Copy from S3 to HDFS

In [17]:
t1=time()
s3helper.s3_to_hdfs('CVS', 'CVS')
time()-t1

487.370493888855

### Read data into dataframe

In [18]:
from pyspark.sql import DataFrame
CVS_Data=sc.textFile("/CVS/")
RDD=CVS_Data.map(parse)

df=sqlContext.createDataFrame(RDD)

t0=time()
print df.cache().count()
print time()-t0

t0=time()
print df.count()
time()-t0

In [20]:
sc.stop()

6353182


0.8903360366821289